In [0]:
%pip install -U transformers
dbutils.library.restartPython()

In [0]:
import pyspark.sql.functions as F
from transformers import pipeline
import json



In [0]:
num_workers = 4
transcriptions_table = spark.table("marcell.call_centre_processing.transcriptions_redacted")

transcriptions_table = transcriptions_table.repartition(num_workers).cache()

transcriptions_table.display()

In [0]:
import pandas as pd
from typing import Iterator

In [0]:
@F.pandas_udf("string")
def classify_udf(text_iterator: Iterator[pd.Series]) -> Iterator[pd.Series]:
    import torch
    from transformers import pipeline

    pipeline = pipeline(
        "text-generation",
        model="microsoft/phi-4",
        model_kwargs={"torch_dtype": "auto"},
        device_map="auto",
    )


    def classify_text(text: str) -> str:

        messages = [
            {"role": "system", "content": "You are an expert at determining the underlying category of a short text passage. Your input is a short text passage and your output is a category. Do not output anything else but one of the following categories that best fits the text passage: 'Politics', 'Sports', 'Entertainment', 'Technology', 'Personal', 'Other'."},
            {"role": "user", "content": text},
        ]

        outputs = pipeline(messages, max_new_tokens=128)
        return outputs[0]["generated_text"][-1]['content']

    for text_chunk in text_iterator:
        yield text_chunk.apply(classify_text)

In [0]:

transcriptions_table.withColumn("transcrtiption_class", classify_udf("redacted_transcription")).write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("marcell.call_centre_processing.transcriptions_classified")